## Deliverable 2. Create a Customer Travel Destinations Map.

In [32]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [33]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Plettenberg Bay,-34.0527,23.3716,63.07,82,100,4.72,ZA,light rain
1,1,Hithadhoo,-0.6000,73.0833,81.73,74,100,10.11,MV,overcast clouds
2,2,Punta Arenas,-53.1500,-70.9167,42.91,61,75,6.91,CL,broken clouds
3,3,Vaini,-21.2000,-175.2000,84.36,84,75,13.80,TO,broken clouds
4,4,Raga,8.4596,25.6780,70.81,22,100,4.45,SS,overcast clouds


In [38]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
#ask the customer for the minimum and maximum temperature values
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 55
What is the maximum temperature you would like for your trip? 70


In [39]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

filtered_cities_df.head()

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Plettenberg Bay,-34.0527,23.3716,63.07,82,100,4.72,ZA,light rain
7,7,Bredasdorp,-34.5322,20.0403,59.29,77,100,4.61,ZA,overcast clouds
11,11,Castro,-24.7911,-50.0119,64.17,98,100,4.05,BR,overcast clouds
13,13,Kasongo-Lunda,-6.4667,16.8167,68.94,98,100,1.83,CD,overcast clouds
18,18,Salalah,17.0151,54.0924,69.89,78,0,2.30,OM,clear sky


In [40]:
# 4a. Determine if there are any empty rows.
filtered_cities_df.count()

City ID                118
City                   118
Lat                    118
Lng                    118
Max Temp               118
Humidity               118
Cloudiness             118
Wind Speed             118
Country                115
Current Description    118
dtype: int64

In [41]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
filtered_cities_df.dropna()

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Plettenberg Bay,-34.0527,23.3716,63.07,82,100,4.72,ZA,light rain
7,7,Bredasdorp,-34.5322,20.0403,59.29,77,100,4.61,ZA,overcast clouds
11,11,Castro,-24.7911,-50.0119,64.17,98,100,4.05,BR,overcast clouds
13,13,Kasongo-Lunda,-6.4667,16.8167,68.94,98,100,1.83,CD,overcast clouds
18,18,Salalah,17.0151,54.0924,69.89,78,0,2.30,OM,clear sky
...,...,...,...,...,...,...,...,...,...,...
672,672,Charleston,32.7766,-79.9309,69.03,91,75,5.01,US,broken clouds
674,674,Kamina,-8.7386,24.9906,62.83,97,100,2.04,CD,overcast clouds
677,677,Sokoto,13.0609,5.2390,59.63,18,0,16.49,NG,clear sky
681,681,Garowe,8.4054,48.4845,67.71,40,2,9.91,SO,clear sky


In [42]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = filtered_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Plettenberg Bay,ZA,63.07,light rain,-34.0527,23.3716,
7,Bredasdorp,ZA,59.29,overcast clouds,-34.5322,20.0403,
11,Castro,BR,64.17,overcast clouds,-24.7911,-50.0119,
13,Kasongo-Lunda,CD,68.94,overcast clouds,-6.4667,16.8167,
18,Salalah,OM,69.89,clear sky,17.0151,54.0924,
19,Coahuayana,MX,67.19,scattered clouds,18.7333,-103.6833,
21,Simao,CN,68.27,clear sky,22.7886,100.9748,
23,Pisco,PE,69.85,clear sky,-13.7000,-76.2167,
27,Praia Da Vitoria,PT,57.45,few clouds,38.7333,-27.0667,
31,Cape Town,ZA,61.88,few clouds,-33.9258,18.4232,


In [43]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        hotel_df.loc[index, "Hotel Name"] = None
        print("Hotel not found, skipping...")
        

Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...


In [44]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Plettenberg Bay,ZA,63.07,light rain,-34.0527,23.3716,Thanda Vista B&B
7,Bredasdorp,ZA,59.29,overcast clouds,-34.5322,20.0403,Bredasdorp Country Manor
11,Castro,BR,64.17,overcast clouds,-24.7911,-50.0119,CHACARA BAILLY
13,Kasongo-Lunda,CD,68.94,overcast clouds,-6.4667,16.8167,None
18,Salalah,OM,69.89,clear sky,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
19,Coahuayana,MX,67.19,scattered clouds,18.7333,-103.6833,El rranchito michuacan de ocampos
21,Simao,CN,68.27,clear sky,22.7886,100.9748,Changyu Hotel
23,Pisco,PE,69.85,clear sky,-13.7000,-76.2167,Hotel San Isidro Oficial
27,Praia Da Vitoria,PT,57.45,few clouds,38.7333,-27.0667,Hotel Branco I
31,Cape Town,ZA,61.88,few clouds,-33.9258,18.4232,Southern Sun Waterfront Cape Town


In [45]:
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Plettenberg Bay,ZA,63.07,light rain,-34.0527,23.3716,Thanda Vista B&B
7,Bredasdorp,ZA,59.29,overcast clouds,-34.5322,20.0403,Bredasdorp Country Manor
11,Castro,BR,64.17,overcast clouds,-24.7911,-50.0119,CHACARA BAILLY
18,Salalah,OM,69.89,clear sky,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
19,Coahuayana,MX,67.19,scattered clouds,18.7333,-103.6833,El rranchito michuacan de ocampos


In [46]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [49]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} *F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [50]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat","Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))